![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Drivers

## What is a `Driver`?!

A `Driver` object represents a simulation you want to run on your `System`. This can be the mathematical
resolution of non-linear system or an optimization for example.


## Introduction

### Add a `Driver`

Simply use the `add_driver` method passing the `Driver` object you want to use 
(see [Available drivers](#Available-Drivers) section of this tutorial).

In [ ]:
from cosapp.tests.library.systems import Multiply1
from cosapp.drivers import RunOnce

m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))

`Multiply1` *is the same* `System` *as* `Multiply` *in the* [Systems](01-Systems.ipynb) *tutorial*

 ![Driver in system](images/drivers_1.svg)

### Implementation

Every `System` can have one or multiple `Driver` objects. They are stored in the `drivers` attribute.
By default, there is no `Driver` attached to a `System`.

The `run_drivers` method of `System` executes its drivers. In the following example, the simplest driver
[RunOnce](#RunOnce) will be added and then executed.

In [ ]:
from cosapp.tests.library.systems import Multiply1

m = Multiply1('mult')
m.add_driver(RunOnce('run'))
print(m.drivers) # print drivers of the system

m.p_in.x = 15.
m.K1 = 2.
m.run_drivers()
print(m.p_out.x)

### Subdrivers - Similarities with `System`

A `Driver` can have children which also inherit from `Driver`, they are kind of numerical systems. It allows more complex simulation such as workflow, multipoints design, design of experiments, optimization, etc.

By construction, a `System` can have n-levels of drivers. These children are stored in the `children` attribute of `Drivers`. Use the `add_child` method of a `Driver`.


![drivers](images/drivers_2.svg)


In [ ]:
m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))

print(run.children) # the 'run' driver has no child

subrun = run.add_child(RunOnce('subrun')) # add a child `Driver` called 'subrun'
print(run.children, subrun.children)

## Available Drivers

**CoSApp** comes with a bunch of `Drivers` to allow users building their simulations

### RunOnce

It makes your `System` and its subsystems compute their code. It does not deal with residues or iterative loops that may be necessary to resolve the `System`.

Let's define a new `System` that has a residue for the purpose of this section.

In [ ]:
from cosapp.systems import System
from cosapp.tests.library.ports import XPort

class MultiplyWithResidue(System):

    def setup(self):
        self.add_input(XPort, 'p_in', {'x': 1.})
        self.add_inward('K1', 5.)
        # expected_output will be used as a target for p_out.x variable
        self.add_inward('expected_output', 7.5)
        self.add_output(XPort, 'p_out', {'x': 1.})
        
        # create a new equation: expected_output = p_out.x
        self.add_equation('expected_output == p_out.x') 

    def compute(self):
        self.p_out.x = self.p_in.x * self.K1


Then test this `Driver`

In [ ]:
from cosapp.drivers import RunOnce

m = MultiplyWithResidue('mult')
run = m.add_driver(RunOnce('run', verbose=True))

m.run_drivers()

print('List of defined drivers\n', m.drivers)
print('\np_in.x\tK1\tp_out.x\tresidue')
print('{}\t{}\t{}\t{}'.format(m.p_in.x, m.K1, m.p_out.x, m.residues.values()))

### NonLinearSolver

It resolves your `System` residues playing with free parameters. This `Driver` resolves the mathematical problem between free parameters and residues of his children drivers.
By default, it comes with a [RunSingleCase](#RunSingleCase) child, called *runner*. 

Note : this `Driver` does not compute the `System`. So the recommended structure is to add a `RunSingleCase` driver to
a non-linear solver. And then to customized it to be able to solve the system.

![scipysolver](images/drivers_nonlinear.svg)


In [ ]:
from cosapp.drivers import NonLinearSolver

m = MultiplyWithResidue('mult')
solver = m.add_driver(NonLinearSolver('solver', verbose=True))
solver.runner.offdesign.add_unknown('p_in.x')
m.run_drivers()

print(m.drivers)
print(m.drivers['solver'].children)
print(m.p_in.x, m.K1, m.expected_output, m.p_out.x, m.residues.values())

### RunSingleCase

It executes all subsystems drivers by calling `compute` method of each `children`. It also computes the `System` itself if it has code defined in the `compute` method.

This `Driver` does not have a solver but is helpfull to set boundary conditions, initial values and additional equations (see [Advanced Drivers](./03b-Advanced-Drivers.ipynb) tutorial).

To update its `System` owner, 4 possibilities are offered:

- *value modification* through `set_values` method will change an input value
- *initial value definition* through `set_init` method will change the value of iteratives
before resolving the case.

In [ ]:
from cosapp.drivers import RunSingleCase

m = MultiplyWithResidue('mult')
update = m.add_driver(RunSingleCase('update', verbose=True))
update.offdesign.add_unknown('p_in.x')

update.set_values({'expected_output': 15.})
update.set_init({'K1': 2.})
m.run_drivers()

print('List of defined drivers\n', m.drivers)
print('\np_in.x\tK1\tp_out.x\tresidue')
print('{}\t{}\t{}\t{}'.format(m.p_in.x, m.K1, m.p_out.x, m.residues.values()))

![updatesystem](images/drivers_4.svg)

In [ ]:
m.drivers.clear() # Remove all drivers on the system `m`
solver = m.add_driver(NonLinearSolver('solver', verbose=True))
update = solver.add_child(RunSingleCase('update', verbose=True))
update.offdesign.add_unknown('p_in.x')

# Customization of the case
update.set_values({'expected_output': 15.})

# Execution
m.run_drivers()

print(m.drivers)
print(m.p_in.x, m.K1, m.expected_output, m.p_out.x, m.residues.values())

**Congrats!** You are now ready to launch computation on your `System` with **CoSApp**!

But we recommend you to read the [Advanced Drivers](./03b-Advanced-Drivers.ipynb) tutorial, before 
creating your test case.